# Long Short Term memory and other Neural Networks

In [1]:
import duckdb
import pandas as pd
import pandas_ta as ta
import os
import csv
import itertools


def trail_stoploss(row, tradingdata, whole_percentage):
    idx = row.name
    td = tradingdata
    pct = whole_percentage/100
    max_holdtime = 30 # Only hold the trade for twenty minutes

    after_td = td.loc[idx:idx+max_holdtime+1] 
    
    basis = td['close'].loc[idx]
    max_price = basis   # Initially the max price is the basis - No Shorting to Start
    min_price = basis - (basis*pct)
    time_stop = idx + max_holdtime 
    time_counter = idx
    
    #print(f"Basis: {basis}")

    for i, row in after_td.iterrows():
        close = row['close']
        #print(f"Close: {close}, Max: {max_price}, Min: {min_price}, Counter: {time_counter-idx}")
        if close > max_price:
            max_price = close
            min_price = max_price - (max_price * pct)
        elif close < min_price:
            profit = (close - basis) / basis
            return [i, profit]
        
        time_counter += 1
        if time_counter > time_stop:
            break

    # If the loop ends without triggering stop loss, calculate the profit based on the last close
    profit = (close - basis) / basis
    return profit


con = duckdb.connect(r"C:\Users\rybot\OneDrive\Databases\datadump.duckdb")

# Filter data for January to March 2022
start_date = '2022-01-01'
end_date = '2022-03-31'

qry_month = f'''
    SELECT * FROM Stocks
    WHERE Datetime >= '{start_date}' 
    AND Datetime < '{end_date}' 
    AND Stock = 'AAPL' 
    AND Interval = 1
'''

df = con.execute(qry_month).fetchdf()
# Sort the Data from oldest to newest
df = df.sort_values(by='Datetime').reset_index(drop=True)

# List of Testing Variables
Data_order = ["Random","Sorted"] # *combination[3]
#Underlying_data = ["Percent Change","Raw Data"]
Field_size = [10,20,50,100,200,300] # *combination[2]
Derivatives = [[],["dydx"],["dydx","dydx2"]] # *combination[0]
TAs = [[],["EMA"],["EMA","SMA"]] # *combination[1]

all_combinations = itertools.product(Derivatives, TAs, Field_size, Data_order)

# Getting all of the trading days because trades can only occur during trading day
df['Time'] = df['Datetime'].dt.time
start_time = pd.to_datetime('09:00:00').time()
end_time = pd.to_datetime('16:00:00').time()
TradingDay_df = df[(df['Time'] > start_time) & (df['Time'] < end_time) & (df.index >= 201)]

df['Profit'] = TradingDay_df.apply(trail_stoploss, axis=1, tradingdata=df,whole_percentage=5)

#applying EMAs SMAs
EMAs = [8,10,20,50,75,100]
SMAs = [8,10,20,50,75,100]


for EMA in EMAs:
    df[f'EMA{EMA}'] = ta.ema(df['close'], length=EMA)
    df[f'EMA{EMA}_pct'] = df[f'EMA{EMA}'].pct_change()
    df = df.drop([f'EMA{EMA}'],axis=1).copy()

for SMA in SMAs:
    df[f'SMA{SMA}'] = ta.sma(df['close'], length=SMA)
    df[f'SMA{SMA}_pct'] = df[f'SMA{SMA}'].pct_change()
    df = df.drop([f'SMA{SMA}'],axis=1).copy()

# 1st (pct chg)
df['dydx'] = df['close'].pct_change()

# 2nd
df['dydx2'] = df['dydx'].pct_change()

# Pytrorch CNN

### 1. Prepare Your Data
- 

In [ ]:
import torch
import numpy as np

# Assuming df is your dataframe
data = df.to_numpy()

# Convert to tensor and reshape: (batch_size, 1, 100, features)
data_tensor = torch.tensor(data).float().reshape(-1, 1, 100, data.shape[1])


In [ ]:
from torch.utils.data import DataLoader, Dataset

class StockDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Create dataset and dataloader
dataset = StockDataset(data_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class StockCNN(nn.Module):
    def __init__(self):
        super(StockCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1)
        self.fc1 = nn.Linear(32 * 100 * (data.shape[1] // 4), 128)  # Adjust dimensions as needed
        self.fc2 = nn.Linear(128, 1)  # 1 output for regression or classification

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (2, 2))  # Reduce dimensions
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, (2, 2))
        x = x.view(x.size(0), -1)  # Flatten before feeding into fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model = StockCNN()

# Loss function for regression (use MSE for regression or BCE for binary classification)
criterion = nn.MSELoss()

# Optimizer (e.g., Adam for better convergence)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs in dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)  # Targets need to be prepared
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader)}")

In [ ]:
model.eval()  # Switch to evaluation mode
with torch.no_grad():
    for inputs in validation_loader:
        outputs = model(inputs)
        # Compare outputs with true values
